In [1]:
import pandas as pd

In [3]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
food = pd.read_csv("Restaurant - Foods.csv")
satis = pd.read_csv("Restaurant - Week 1 Satisfaction.csv")

In [5]:
# combine week 1 and week 2 sales
pd.concat(objs = [week1, week2])
# you can ignore previous indices using ignore_index

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [6]:
week1.append(week2)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


We can use multiindex to keep track of where the data came from

In [8]:
sales = pd.concat(objs = [week1, week2], keys=["Week 1", "Week 2"])
sales

Customer ID  Food ID
Week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
Week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

In [9]:
sales.loc[("Week 1",)]

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6
